**Here we will be building a movie recommendation system based collaborative filtering.**



1. Importing Libraries

For this project we will be importing numpy, pandas, scikit learn, matplotlib and seaborn.

In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

2. Loading Datasets

Here we will be importing rating dataset that contains ratings of movies. You can download the dataset "ratings.csv".

In [2]:
ratings = pd.read_csv("ratings.csv")
print(ratings.head())

   userId  movieId  rating   timestamp
0     877     4155       5  1651201566
1     305     7661       2  1639553712
2     381     8423       2  1610704432
3     208     6433       1  1650223767
4      47     7752       4  1663998365


Now we will import movies dataset that contains movie ID, Titles and Genres based on which our model will recommend movies. You can download the dataset "movies.csv".

In [3]:
movies = pd.read_csv("movies.csv")
print(movies.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


3. Statistical Analysis of Ratings

Now we will calculate and prints the total number of ratings, unique movies, unique users and the average number of ratings per user and per movie.It will help us in model making and making informed decision.

*   **len(ratings):** calculates the total number of ratings in the ratings DataFrame.
*   **ratings['movieId'].unique():** gets the unique movie IDs to calculate the total number of unique movies.
*   **ratings['userId'].unique():** gets the unique user IDs to calculate the total number of unique users.
*   **round(n_ratings/n_users, 2):** calculates the average number of ratings per user and rounds it to 2 decimal places.
*   **round(n_ratings/n_movies, 2):** calculates the average number of ratings per movie and rounds it to 2 decimal places.


In [4]:
n_ratings = len(ratings)
n_movies = len(ratings['movieId'].unique())
n_users = len(ratings['userId'].unique())

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per movie: {round(n_ratings/n_movies, 2)}")

Number of ratings: 100836
Number of unique movieId's: 9742
Number of unique users: 999
Average ratings per user: 100.94
Average ratings per movie: 10.35


4. User Rating Frequency

Now we will block groups the ratings dataset by user ID to calculate the number of ratings each user has made and then prints the first few rows of this user rating frequency data.

*   **ratings[['userId', 'movieId']].groupby('userId').count():** groups the ratings by userId and counts the number of movies each user has rated.



In [5]:
user_freq = ratings[['userId', 'movieId']].groupby(
    'userId').count().reset_index()
user_freq.columns = ['userId', 'n_ratings']
print(user_freq.head())

   userId  n_ratings
0       1        120
1       2        105
2       3         89
3       4        100
4       5        107


5. Movie Rating Analysis

This block analyzes the average ratings of movies identifies the highest and lowest rated movies and displays information about those movies from the movies DataFrame.

*   **ratings.groupby('movieId')[['rating']].mean():** groups the ratings by movieId and calculates the mean rating for each movie.
*   **mean_rating['rating'].idxmin():** finds the movie ID with the lowest average rating.
*   **mean_rating['rating'].idxmax():** finds the movie ID with the highest average rating.
*   **movies.loc[movies['movieId'] == lowest_rated]:** filters the movies DataFrame to display information about the movie with the lowest rating.
*   **movies.loc[movies['movieId'] == highest_rated]:** filters the movies DataFrame to display information about the movie with the highest rating.


In [6]:
mean_rating = ratings.groupby('movieId')[['rating']].mean()
lowest_rated = mean_rating['rating'].idxmin()
movies.loc[movies['movieId'] == lowest_rated]
highest_rated = mean_rating['rating'].idxmax()
movies.loc[movies['movieId'] == highest_rated]
ratings[ratings['movieId']==highest_rated]
ratings[ratings['movieId']==lowest_rated]

movie_stats = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

6. User-Item Matrix Creation

This block creates a sparse user-item matrix using csr_matrix from scipy. It also generates mappings between user and movie IDs and their corresponding indices for use in the matrix.

*   **csr_matrix:** creates a sparse matrix (Compressed Sparse Row) from the user-item ratings data to save memory.
*   **user_mapper and movie_mapper:** create dictionaries that map user IDs and movie IDs to indices in the sparse matrix.
*   **user_inv_mapper and movie_inv_mapper:** create reverse dictionaries that map matrix indices back to user IDs and movie IDs.
*   **user_index and movie_index:** create lists of indices for users and movies from the ratings DataFrame.
*   **X = csr_matrix(...):** constructs the sparse user-item matrix X with the given user and movie indices and ratings.


In [7]:
from scipy.sparse import csr_matrix

def create_matrix(df):

    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))

    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))

    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)

7. Movie Similarity Analysis

Here we will use k-nearest neighbors algorithm to find similar movies based on the cosine similarity metric. It calculates the KNN for the given movie ID and returns a list of similar movie IDs.

*   **kNN.fit(X):** fits the k-NN model to the user-item matrix X.
*   **kNN.kneighbors(movie_vec, return_distance=show_distance):** finds the k-nearest neighbors for the given movie using the k-NN algorithm.
*   **neighbour_ids.pop(0):** removes the movie itself from the list of similar movies.


In [8]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    neighbour_ids = []

    if movie_id not in movie_mapper:
        print(f"Movie ID {movie_id} not found in movie_mapper!")
        return []

    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k += 1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1, -1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)

    for i in range(0, k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])

    neighbour_ids.pop(0)
    return neighbour_ids

8. Movie Recommendation with respect to Users Preference

*   This function recommends movies based on a user’s highest-rated movie. It filters the ratings dataset to find the movie with the highest rating for the given user.
*   It then uses the find_similar_movies function to find movies similar to the highest-rated movie.
*   The movie titles are printed as recommendations and any movies that aren't found in the dataset are skipped.


In [9]:
def recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10):
    df1 = ratings[ratings['userId'] == user_id]

    movie_id = df1[df1['rating'] == max(df1['rating'])]['movieId'].iloc[0]

    movie_titles = dict(zip(movies['movieId'], movies['title']))

    similar_ids = find_similar_movies(movie_id, X, k)

    print(f"Since you watched {movie_titles[movie_id]}, you might also like:")

    for i in similar_ids:
        if i in movie_titles:
            print(movie_titles[i])

9. Recommending movies

Here we are testing our model on user id= 150 and will see what models recommend based on user activity.

In [10]:
user_id = 150
recommend_movies_for_user(user_id, X, user_mapper, movie_mapper, movie_inv_mapper, k=10)

Since you watched Miller's Crossing (1990), you might also like:
Flawless (1999)
Lilya 4-Ever (Lilja 4-ever) (2002)
Bells of St. Mary's, The (1945)
Dark City (1998)
Cradle 2 the Grave (2003)
Japanese Story (2003)


We can see that our model is working fine and making predictions based on other user behaviors using KNN.